In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
df_train=pd.read_csv('../input/av-healthcare-analytics-ii/healthcare/train_data.csv')
df_train_dict=pd.read_csv('../input/av-healthcare-analytics-ii/healthcare/train_data_dictionary.csv')
df_test=pd.read_csv('../input/av-healthcare-analytics-ii/healthcare/test_data.csv')
df_samp=pd.read_csv('../input/av-healthcare-analytics-ii/healthcare/sample_sub.csv')

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train.head()

In [ ]:
df_train['Stay'].value_counts()

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

**** data cleaning steps by substituting the null values with some other values.****

In [ ]:
df_train.shape 

In [ ]:
df_test.shape

In [ ]:
cols=df_train.columns
for col in cols:
    print('Number of unique categories for',col,'are:',df_train[col].nunique())

In [ ]:
df_train['case_id'].value_counts().sum()

In [ ]:
import matplotlib.pyplot as plt
plt.subplots(figsize=(15,4))
import seaborn as sns 
sns.countplot(x='Stay',hue='Age',data=df_train)

In [ ]:
plt.subplots(figsize=(15,4))
sns.countplot(x='Stay',hue='Type of Admission',data=df_train)


In [ ]:
plt.subplots(figsize=(15,4))
sns.countplot(x='Stay',hue='Severity of Illness',data=df_train)

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
df_train['train_flag']=1

In [ ]:
df_hos_code=df_train.groupby('Hospital_type_code')['train_flag'].sum().reset_index().sort_values(by='train_flag',ascending=False)


In [ ]:
fig1=px.bar(df_hos_code,x='Hospital_type_code',y='train_flag',color='train_flag',labels={'Hospital_type_code':'Hospital Type Code','train_flag':'Number of patients'})
fig1.update_layout(title='Patient distribution per hospital type code',title_x=0.5,template='plotly_dark')
fig1.show()

In [ ]:
objectColumns = []
for col in df_train:
    if df_train[col].dtype.name == 'object':
        df_train[col] = df_train[col].astype('category')
        df_train[col] = df_train[col].cat.codes
        objectColumns.append(col)

print(objectColumns)

In [ ]:
df_train.head(20)


In [ ]:
import sklearn
from sklearn import preprocessing
import shap
from sklearn.model_selection import train_test_split


In [ ]:
scaler=preprocessing.MinMaxScaler()
scaled=pd.DataFrame(scaler.fit_transform(df_train),columns=df_train.columns)

In [ ]:
X=df_train.iloc[:, : len(df_train.columns)]
Y=df_train.iloc[:,-1:]

Splitting in train and test set

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)
X_trainDframe = X_train
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

Y_test = Y_test.to_numpy()
Y_train = Y_train.to_numpy()


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

model=XGBClassifier(max_depth=4, n_estimators=499, learning_rate=0.1, colsample_bytree=0.78)
model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)
predictions = [round(value) for value in Y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
pip install lime

In [ ]:
!pip install eli5

In [ ]:
columns = ['','case_id', 'Hospital_code', 'Hospital_type_code','City_Code_Hospital', 'Hospital_region_code','Available_Extra_Rooms_in_Hospital','Department', 'Ward_Type','Ward_Facility_Code','Bed_Grade', 'patientid', 'City_Code_Patient','Type_of_Admission', 'Severity_of_Illness', 'Visitors_with_Patient', 'Age', 'Admission_Deposit', 'Stay']

In [ ]:
# Create object that can calculate shap values
explainerShap = shap.TreeExplainer(model)

In [ ]:
row_to_show = 12
data_for_prediction = X_trainDframe.iloc[row_to_show]

shap_values = explainerShap.shap_values(data_for_prediction)
shap.initjs()
shap.force_plot(explainerShap.expected_value, shap_values[0], data_for_prediction)

In [ ]:
import lime
import lime.lime_tabular
results=[]
explainer = lime.lime_tabular.LimeTabularExplainer(X_train, feature_names=columns, class_names=['hospital_patient_staying_status'], verbose=True, mode='regression')

In [ ]:
j = 12
exp = explainer.explain_instance(X_test[j], model.predict, num_features=6)
exp.show_in_notebook(show_table=True)